## Import libraries

In [1]:
# reload(sys)
import pandas as pd
import numpy as np
import os
import random
from pdb import set_trace as bp
import matplotlib.pyplot as plt
from collections import defaultdict
import pickle
import sys 
from sklearn.utils import shuffle
from dataGenerator import DataGenerator
from models import QuoraModel
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence
from utils import plotGraph,plotLengthHistogram,plotHist,labelPlot
from utils import createTiles, plotHistory

Using TensorFlow backend.


In [98]:
from keras.models import Sequential
from keras.layers import Input,Dense, LSTM,TimeDistributed, Flatten, Reshape
from keras.models import Model
import keras.backend as K
from keras.preprocessing import sequence

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint

class QuoraModel:
    
    def __init__(self, timeSlice = 2, questionLen = 25, wordEmbeddDim = 300, dataPath = './model', weightFile = 'model.h5' ):
        self.timeSlice = timeSlice
        self.questionLen = questionLen
        self.wordEmbeddDim = wordEmbeddDim
        self.dataPath = dataPath
        self.model_best_loss = 100
        self.model_best_acc =  0
        self.weightFilePath = os.path.join(dataPath, weightFile )
        
    def getCallbackList(self, model_name = 'model1'):
        fileName= model_name + "_weights-{epoch:02d}-{val_loss:.3f}-{val_acc:.2f}.hdf5"
        filePath = os.path.join(self.dataPath, fileName)
        checkpoint = ModelCheckpoint(filePath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
        callbacks_list = [checkpoint]
        return callbacks_list
        
    def getModel1(self, load_weight = True):
        layer1Dim = 500
        layer2Dim = 250
        nDense1   = 256
        nDense2   = 128
        inputs = Input(shape=(self.timeSlice,self.questionLen, self.wordEmbeddDim))
        batchNorm1 = BatchNormalization(name = "batch_norm_1")(inputs)
        lstm1 = TimeDistributed(LSTM(layer1Dim, activation = 'relu', return_sequences = True, dropout = 0.3), name = 'lstm_1')(batchNorm1)
        batchNorm2 = BatchNormalization( name = "batch_norm_2")(lstm1)
        lstm2 = TimeDistributed(LSTM(layer2Dim, activation = 'relu', return_sequences = False), name = 'lstm_2')(batchNorm2)
        flatten = Flatten(name = 'flatten')(lstm2)
        batchNorm3 = BatchNormalization(name = "batch_norm_3")(flatten)
        dense1 = Dense(nDense1, activation = 'relu')(batchNorm3)
        batchNorm4 = BatchNormalization(name = "batch_norm_4")(dense1)
        dense2 = Dense(nDense2, activation = 'relu')(batchNorm4)
        output = Dense(1, activation = 'sigmoid')(dense2)
        self.model = Model(inputs = inputs, outputs = output)
        if load_weight:
            if os.path.isfile(self.weightFilePath) :
                try:
                    self.model.load_weights(self.weightFilePath)
                    print("Weights loaded")
                except:
                    print("Failed to load weights")
                    pass
        return self.model

    
    def getModel2(self):
        layer1Dim = 500
        layer2Dim = 250
        num_dense   = 100
        rate_drop_lstm = 0.15 + np.random.rand() * 0.25
        rate_drop_dense = 0.15 + np.random.rand() * 0.25
        act = 'relu'
        bp()
        lstm_layer = LSTM(layer1Dim)

        sequence_1_input = Input(shape=(self.questionLen, self.wordEmbeddDim), dtype='int32')
        x1 = lstm_layer(sequence_1_input)
        
        sequence_2_input = Input(shape=(self.questionLen, self.wordEmbeddDim), dtype='int32')
        x2 = lstm_layer(sequence_2_input)

        merged = concatenate([x1, x2])
        merged = Dropout(rate_drop_dense)(merged)
        merged = BatchNormalization()(merged)
        merged = Dense(num_dense, activation=act)(merged)
        merged = Dropout(rate_drop_dense)(merged)
        merged = BatchNormalization()(merged)

        preds = Dense(1, activation='sigmoid')(merged)
        self.model = Model(inputs=[sequence_1_input, sequence_2_input], \
                           outputs=preds)
        
        return self.model
    
#     @staticmethod
#     def saveWeights(self):
#         try:
#             self.model.save_weights(self.weightFilePath)
#             print("Weights successfully saved")
#         except:
#             print("Exception: Weight save failed")

In [92]:
## Define parameters
timeSlice = 2
questionLen = 25
wordEmbeddDim = 100
batchSize  = 16
nEpochs = 10

In [93]:
dataGenerator = DataGenerator(batchSize = batchSize, 
                              questionLen = questionLen,
                              wordEmbeddDim = wordEmbeddDim)
nData = dataGenerator.nTrainData

Data Generator succesfully initialized :) :) :) 


In [109]:
# This returns a tensor
net = QuoraModel(questionLen = questionLen,
                   wordEmbeddDim = wordEmbeddDim)
model = net.getModel1(load_weight = False)
nadam = Nadam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(optimizer=nadam,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 2, 25, 100)        0         
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 2, 25, 100)        400       
_________________________________________________________________
lstm_1 (TimeDistributed)     (None, 2, 25, 500)        1202000   
_________________________________________________________________
batch_norm_2 (BatchNormaliza (None, 2, 25, 500)        2000      
_________________________________________________________________
lstm_2 (TimeDistributed)     (None, 2, 250)            751000    
_________________________________________________________________
flatten (Flatten)            (None, 500)               0         
_________________________________________________________________
batch_norm_3 (BatchNormaliza (None, 500)               2000      
__________

# train Network

## train Model 1

In [110]:
trainHist = [[], []]
validHist = [[], []]
iterSize = 100
bestLoss = 100
weightFilePath = "./1234"
nIter = dataGenerator.nTrainData/(batchSize)
for i in range(nEpochs):
    for j in range(nIter):
        trainX, trainY = dataGenerator.getTrainBatch()
        model.fit( trainX, trainY, verbose = False )
        if( j%iterSize == 0 ):
            trainX, trainY = dataGenerator.getTrainBatch(batchSize = 5*batchSize)
            validX, validY = dataGenerator.getValidBatch(batchSize = 5*batchSize)
            trainLoss, trainAcc = model.evaluate( trainX, trainY, verbose = False )
            validLoss, validAcc = model.evaluate(validX, validY, verbose = False )
            if validLoss < bestLoss:
                bestLoss = validLoss
                print("Best loss:{0:.3f}\t BessAcc: {1:.3f} \t Saving weights ".format(validLoss, validAcc))
                if os.path.isfile(weightFilePath):
                    os.remove(weightFilePath)
                weightFilePath = 'model/weights_loss_{0:.4f}_acc_{1:.3f}.h5py'.format(validLoss, validAcc)
                model.save_weights(weightFilePath)
            print( "Epoch:{0}\t Iter: {1}\t TrainLoss {2:.4f}\t ValidLos: {3:.4f} \
            TrainAcc: {4:.4f}\t ValidAcc: {5:.4f}"
                   .format(i, j+1, trainLoss, validLoss, trainAcc, validAcc))
            trainHist[0].append(trainLoss)
            trainHist[1].append(trainAcc)
            validHist[0].append(validLoss)
            validHist[1].append(validAcc)
plotHistory(trainHist, validHist)

Best loss:0.693	 BessAcc: 0.512 	 Saving weights 
Epoch:0	 Iter: 1	 TrainLoss 0.6926	 ValidLos: 0.6934             TrainAcc: 0.6250	 ValidAcc: 0.5125
Epoch:0	 Iter: 11	 TrainLoss 0.6848	 ValidLos: 0.6935             TrainAcc: 0.7500	 ValidAcc: 0.5250
Best loss:0.689	 BessAcc: 0.600 	 Saving weights 
Epoch:0	 Iter: 21	 TrainLoss 0.6779	 ValidLos: 0.6885             TrainAcc: 0.7500	 ValidAcc: 0.6000
Best loss:0.684	 BessAcc: 0.613 	 Saving weights 
Epoch:0	 Iter: 31	 TrainLoss 0.6729	 ValidLos: 0.6844             TrainAcc: 0.7500	 ValidAcc: 0.6125
Best loss:0.680	 BessAcc: 0.650 	 Saving weights 
Epoch:0	 Iter: 41	 TrainLoss 0.6695	 ValidLos: 0.6799             TrainAcc: 0.7500	 ValidAcc: 0.6500
Epoch:0	 Iter: 51	 TrainLoss 0.6660	 ValidLos: 0.6834             TrainAcc: 0.7500	 ValidAcc: 0.6125
Epoch:0	 Iter: 61	 TrainLoss 0.6627	 ValidLos: 0.6800             TrainAcc: 0.7500	 ValidAcc: 0.6250
Epoch:0	 Iter: 71	 TrainLoss 0.6607	 ValidLos: 0.6869             TrainAcc: 0.7500	 ValidAcc: 

KeyboardInterrupt: 

## train Model 2

In [61]:
# trainHist = [[], []]
# validHist = [[], []]
# model2 = False
# nIter = dataGenerator.nTrainData/(batchSize)
# for i in range(nEpochs):
#     for j in range(nIter):
#         trainX, trainY = dataGenerator.getTrainBatch()
#         if model2:
#             trainX = [trainX[:,0], trainX[:,1]]
#         model.fit( trainX, trainY, verbose = False )
#         if( j%iterSize == 0 ):
#             trainX, trainY = dataGenerator.getTrainBatch(batchSize = 5*batchSize)
#             validX, validY = dataGenerator.getValidBatch(batchSize = 5*batchSize)
#             if model2:
#                 trainX = [trainX[:,0], trainX[:,1]]
#                 validX = [validX[:,0], validX[:,1]]
#             trainLoss, trainAcc = model.evaluate( trainX, trainY, verbose = False )
#             validLoss, validAcc = model.evaluate(validX, validY, verbose = False )
#             print( "Epoch:{0}\t Iter: {1}\t TrainLoss {2:.4f}\t ValidLos: {3:.4f} \
#             TrainAcc: {4:.4f}\t ValidAcc: {5:.4f}"
#                    .format(i, j+1, trainLoss, validLoss, trainAcc, validAcc))
#             trainHist[0].append(trainLoss)
#             trainHist[1].append(trainAcc)
#             validHist[0].append(validLoss)
#             validHist[1].append(validAcc)
#     model.save_weights('data/model.h5')
# plotHistory(trainHist, validHist)